In [16]:
import numpy as np
import cv2
from matplotlib import pyplot as plt


In [10]:
np.set_printoptions(formatter={'float': '{: 0.3f}'.format})

def dlt(originals, correspondence):  ## numpy arrays of the same size
    points_number = np.shape(originals)[0];
    ## o =  orignals
    ## c = correspondence
    o_centroid = np.mean(originals, axis=0)
    c_centroid = np.mean(correspondence, axis=0)
    
    o_centered = originals - np.tile(o_centroid, (points_number,1))
    c_centered = correspondence - np.tile(c_centroid, (points_number,1))
    
    o_norms = np.linalg.norm(o_centered, axis=1)
    c_norms = np.linalg.norm(c_centered, axis=1)                                           
    
    o_distance = np.mean(o_norms)
    c_distance = np.mean(c_norms)
                                          
    o_scale = np.sqrt(2)/o_distance
    c_scale = np.sqrt(2)/c_distance
    
    H_sim_prim = np.array([[c_scale,0      ,-c_centroid[0]*c_scale],
                          [0      ,c_scale,-c_centroid[1]*c_scale],
                          [0      ,0      ,1]])

    
    H_sim = np.array([[o_scale,0      ,-o_centroid[0]*o_scale],
                      [0      ,o_scale,-o_centroid[1]*o_scale],
                      [0      ,0      ,1]])
    
    o_homologous = np.transpose(H_sim @ np.transpose(originals))
    c_homologous = np.transpose(H_sim_prim @ np.transpose(correspondence))

    H_sim_prim = np.linalg.inv(H_sim_prim)
                                             
    A = np.zeros((points_number*2,9))

    zerot = np.array([0,0,0])

    for i in range(0, points_number):
        xt = o_homologous[i, :]
        
        wixt = c_homologous[i,2] * xt
        yixt = c_homologous[i,1] * xt
        xixt = c_homologous[i,0] * xt
        A[2*i,:] = np.concatenate((zerot, -wixt, yixt))
        A[2*i + 1,:] = np.concatenate((wixt, zerot, -xixt))
    (u,d,v) = np.linalg.svd(A)
    
    H = np.reshape(v[-1, :], (3,3))
    H = np.dot(H, H_sim)
    H = np.dot(H_sim_prim, H)
    
    return H

In [ ]:
img = cv2.imread('simple.jpg')
gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)

corners = cv2.goodFeaturesToTrack(gray,25,0.01,10)
corners = np.int0(corners)

for i in corners:
    x,y = i.ravel()
    cv2.circle(img,(x,y),3,255,-1)

plt.imshow(img),plt.show()


In [15]:
alphas = {0.9: 4.605, 0.95:5.991, 0.99:9.210}
desvios = [1, 0.5, 1.5]
t_estandar = alphas[0.95]*desvios[1]
sample_size = 2
#points is a np.ndarray

def create_homografic(originals, correspondences):
    ab_ind = np.random.choice(originals.shape[0], size = 2, replace = False) 
    a_ind = ab_ind[0]
    b_ind = ab_ind[1]
    colineal = True
    a = originals[a_ind,:]
    b = originals[b_ind,:]
    ab = np.cross(a,b)
    c = a
    c_ind = a_ind
    while np.cross(ab, c) == 0:
        c_ind = np.random.choice(originals.shape[0])
        c = originals[c_ind, :]
    ac = np.cross(a,c)
    bc = np.cross(b,c)
    d = a
    d_ind = a_ind
    while np.cross(ab, d) == 0 or np.cross(bc, d) == 0 or np.cross(ac, d) == 0:
        d_ind = np.random.choice(originals.shape[0])
        d = originals[d_ind, :]
        
    return dlt(originals[[a_ind, b_ind, c_ind, d_ind]], correspondences[[a_ind, b_ind, c_ind, d_ind]])

    
    
def d_trans(H, original, correspondence):
    inverse = np.linalg.inv(H)
    d = np.linalg.norm(original, inverse @ correspondence)**2
    d += np.linalg.norm(correspondence, H @ original)
    
    return d

def ransac(originals, correspondences):
    t = 5.991 #desvio es 1
    n = 11 #outliers = 40%
    stoping_size = len(point)*0.6
    sample_size = 2
    H = np.array([])
    support = []
    points_idx = list(range(points.shape()[0]))
    
    for i in range(n):
        H_aux = create_homografic(originals, correspondences)
        cant = 0
        suport_aux = []
        for j in range(points.shape()[0]):
            distance = d_trans(H_aux, originals[j,:], correspondencess[j,:])
            if distance <= t:
                support_aux.append(j)
        if len(support) < len(support_aux):
                support = support_aux
                H = H_aux
        if cant >= stoping_size:
            break

    H_big = dlt(originals[support], correspondences[support])
    
    return H_big

In [9]:
aux[[1,2]]

array([[1, 2, 4],
       [1, 2, 5]])